# Agent  A:  Database  Selector:
`Chooses  the  most  relevant  database  based  on  the  user’s  natural 
language query.`

## Table of Contents
1. [OpenAI Setup](#OpenAI-Setup)
2. [Create Schema Embeddings](#Create-Schema-Embeddings)
3. [Database Selector](#database-selector)


Imports:

In [ ]:
import sys
import json
import openai
import math
import pandas as pd
from src import config
from pathlib import Path
from typing import Dict, Any, List, Tuple
from collections import defaultdict
from datetime import datetime, timezone

In [101]:
PROJECT_ROOT = Path(__file__).parent.parent.parent if '__file__' in globals() else Path.cwd().parent.parent.parent  
sys.path.append(PROJECT_ROOT)

---

## OpenAI Setup

Import OpenAI API Key

In [ ]:
print("OpenAI API key used:", config.OPENAI_API_KEY[:5] + "****")

OpenAI API key used: sk-pr****


make a simple chat completion request

In [21]:
# set your API key from .env
openai.api_key = config.OPENAI_API_KEY

response = openai.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "Hello!"}]
)

# print the model's reply
print(response.choices[0].message.content)

Hello! How can I assist you today?


---

## Create Schema Embeddings

In [103]:
EMBED_MODEL = "text-embedding-3-small"  # OpenAI model name

# Create output directory for embeddings
EMBEDDINGS_OUTPUT_DIR = PROJECT_ROOT / "data" / "processed" / "embeddings"
EMBEDDINGS_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
EMBEDDINGS_FILE = EMBEDDINGS_OUTPUT_DIR / "spider_embeddings.pkl"

SCHEMA_PATH = PROJECT_ROOT / "data" / "spider_data" / "tables.json"
TEST_PATH = PROJECT_ROOT / "data" / "spider_data" / "test.json"

In [104]:
# read schema file
def load_schemas(path: Path) -> Dict[str, Dict[str, Any]]:
    """
    Load the schema file from the given path and return a dictionary of schema objects.
    """
    with open(path, "r", encoding="utf-8") as f:
        items = json.load(f)
    return {it["db_id"]: it for it in items}


schemas = load_schemas(SCHEMA_PATH)
db_ids = list(schemas.keys())

print(db_ids)

['perpetrator', 'college_2', 'flight_company', 'icfp_1', 'body_builder', 'storm_record', 'pilot_record', 'race_track', 'academic', 'department_store', 'music_4', 'insurance_fnol', 'cinema', 'decoration_competition', 'phone_market', 'store_product', 'assets_maintenance', 'student_assessment', 'dog_kennels', 'music_1', 'company_employee', 'farm', 'solvency_ii', 'city_record', 'swimming', 'flight_2', 'election', 'manufactory_1', 'debate', 'network_2', 'local_govt_in_alabama', 'climbing', 'e_learning', 'scientist_1', 'ship_1', 'entertainment_awards', 'allergy_1', 'imdb', 'products_for_hire', 'candidate_poll', 'chinook_1', 'flight_4', 'pets_1', 'dorm_1', 'journal_committee', 'flight_1', 'medicine_enzyme_interaction', 'local_govt_and_lot', 'station_weather', 'shop_membership', 'driving_school', 'concert_singer', 'music_2', 'sports_competition', 'railway', 'inn_1', 'museum_visit', 'browser_web', 'baseball_1', 'architecture', 'csu_1', 'tracking_orders', 'insurance_policies', 'gas_company', 'e_

In [114]:
def schema_text(db: Dict[str, Any], max_cols_per_table: int = 100) -> str:
    """
    Convert a database schemas to a text string with tables and attributes.
    """
    tnames = db["table_names_original"]
    cols = db["column_names_original"]  
    by_table = defaultdict(list)

    for _, (tidx, cname) in enumerate(cols):
        if tidx >= 0:  
            by_table[tidx].append(str(cname))
    parts = []
    for i, t in enumerate(tnames):
        c = by_table[i][:max_cols_per_table]
        cols_txt = ", ".join(c)
        parts.append(f"{t}({cols_txt})")
    # Each table on a new line
    return "\n".join(parts)

db_texts = [schema_text(schemas[i]) for i in db_ids]

print(db_texts[1]) # print the schema for one database
#print(len(db_texts))

classroom(building, room_number, capacity)
department(dept_name, building, budget)
course(course_id, title, dept_name, credits)
instructor(ID, name, dept_name, salary)
section(course_id, sec_id, semester, year, building, room_number, time_slot_id)
teaches(ID, course_id, sec_id, semester, year)
student(ID, name, dept_name, tot_cred)
takes(ID, course_id, sec_id, semester, year, grade)
advisor(s_ID, i_ID)
time_slot(time_slot_id, day, start_hr, start_min, end_hr, end_min)
prereq(course_id, prereq_id)


In [106]:
def batch(iterable, n=64):
    """Split the database schema texts into size n batches of schemas for batching API calls.
    This reduces the # of API calls which makes calls faster and cheaper"""
    it = list(iterable)
    for i in range(0, len(it), n):
        yield it[i:i+n]

def build_texts(schemas: Dict[str, Dict[str, Any]]) -> Dict[str, str]:
    """Creates a dictionary of schemas to their tables(...columns...)"""
    return {db_id: schema_text(db) for db_id, db in schemas.items()}

print(build_texts(schemas))

{'perpetrator': 'perpetrator(Perpetrator_ID, People_ID, Date, Year, Location, Country, Killed, Injured)\npeople(People_ID, Name, Height, Weight, Home Town)', 'college_2': 'classroom(building, room_number, capacity)\ndepartment(dept_name, building, budget)\ncourse(course_id, title, dept_name, credits)\ninstructor(ID, name, dept_name, salary)\nsection(course_id, sec_id, semester, year, building, room_number, time_slot_id)\nteaches(ID, course_id, sec_id, semester, year)\nstudent(ID, name, dept_name, tot_cred)\ntakes(ID, course_id, sec_id, semester, year, grade)\nadvisor(s_ID, i_ID)\ntime_slot(time_slot_id, day, start_hr, start_min, end_hr, end_min)\nprereq(course_id, prereq_id)', 'flight_company': 'airport(id, City, Country, IATA, ICAO, name)\noperate_company(id, name, Type, Principal_activities, Incorporated_in, Group_Equity_Shareholding)\nflight(id, Vehicle_Flight_number, Date, Pilot, Velocity, Altitude, airport_id, company_id)', 'icfp_1': 'Inst(instID, name, country)\nAuthors(authID, l

In [107]:
def embed_texts(texts: List[str], model: str) -> List[List[float]]:
    all_vecs = []
    for chunk in batch(texts, n=64):
        resp = openai.embeddings.create(model=model, input=chunk)
        all_vecs.extend([d.embedding for d in resp.data])
    return all_vecs

vecs = embed_texts(db_texts, EMBED_MODEL)

print("Embedding dimension:", len(vecs[0]) if vecs else 0)
print("Number of embeddings:", len(vecs)) # equal to number of database schemas
print("First embedding:")
print(vecs[0])



Embedding dimension: 1536
Number of embeddings: 166
First embedding:
[0.019305652007460594, 0.01294891256839037, 0.034968260675668716, 0.030903911218047142, -0.02441086806356907, -0.034076087176799774, -0.0031814672984182835, -0.01982608623802662, 0.024485215544700623, -0.004888369236141443, 0.008865977637469769, -0.0008015624480322003, 0.01799217239022255, -0.016740651801228523, 0.021932607516646385, 0.0046095652505755424, 0.04051956534385681, 0.04019739106297493, -0.02137499861419201, 0.012886956334114075, 0.026542173698544502, 0.04354304075241089, -0.005480054300278425, -0.044955648481845856, -0.0013343885075300932, 0.03757043182849884, -0.011344238184392452, 0.03206869587302208, 0.032688260078430176, -0.014237607829272747, 0.019553476944565773, -0.030779998749494553, -0.012100107967853546, -0.03219260647892952, 0.05063086748123169, -0.0212263036519289, 0.046318694949150085, 0.0020151357166469097, 0.006685108412057161, -0.004519728012382984, 0.028946084901690483, -0.0402221716940403

In [108]:
def main():
    schemas = load_schemas(SCHEMA_PATH)
    db_ids = list(schemas.keys())
    db_texts = [schema_text(schemas[i]) for i in db_ids]

    vecs = embed_texts(db_texts, EMBED_MODEL)
    dims = len(vecs[0]) if vecs else 0
    now = datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")

    with open(EMBEDDINGS_FILE, "w", encoding="utf-8") as f:
        for db_id, text, emb in zip(db_ids, db_texts, vecs):
            rec = {
                "db_id": db_id,
                "text": text,
                "embedding": emb,
                "model": EMBED_MODEL,
                "dims": dims,
                "updated_at": now,
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    print(f"Finished creating {len(vecs)} embeddings (dimension: {dims})")
    print(f"Saved to: /PROJECT_ROOT/{EMBEDDINGS_FILE.relative_to(PROJECT_ROOT)}")

if __name__ == "__main__":
    main()


Finished creating 166 embeddings (dimension: 1536)
Saved to: /PROJECT_ROOT/data/processed/embeddings/spider_embeddings.pkl


----

## Database Selector

In [109]:
def load_schema_embeddings(path: str) -> List[Dict]:
    """Load schema embeddings and normalise them. The normalised embedding vector is useful for cosine similarity calcualtions."""
    items = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip() # remove whitespace/newlines
            if line:
                items.append(json.loads(line))
    # Normalise each embedding vector so we can compute cosine similarity
    for rec in items:
        v = rec["embedding"] # original embedidng vector
        n = math.sqrt(sum(x*x for x in v)) + 1e-12 # compute norm and add small epsilon to ensure no divide by 0
        rec["_norm"] = [x / n for x in v] # create normalise vector
    return items

SCHEMAS = load_schema_embeddings(EMBEDDINGS_FILE)

# Access normalized vector of first schema
print(SCHEMAS[0]["_norm"][:10])  # first 10 values
#print(json.dumps(SCHEMAS[0], indent=2))

[0.01930565113244839, 0.012948911981491954, 0.034968259090762135, 0.030903909817353734, -0.0244108669571673, -0.034076085632330194, -0.0031814671542209926, -0.019826085339426175, 0.02448521443492912, -0.004888369014580278]


In [110]:
def embed_query(text: str) -> List[float]:
    """Takes the input query and returns an embedding vector for the query"""
    resp = openai.embeddings.create(model=EMBED_MODEL, input=[text])
    return resp.data[0].embedding

def cosine(u: List[float], v: List[float]) -> float:
    """Computes the cosine similarity between two vectors"""
    nu = math.sqrt(sum(x*x for x in u)) + 1e-12
    uhat = [x/nu for x in u]
    return sum(x*y for x, y in zip(uhat, v))


def select_db(question: str, top_k: int = 3) -> List[Tuple[str, float]]:
    """ Takes a query, computes the embedding vector, compares it with the database schemas' 
    nomralised embedding using cosing similarity, returns a list of the top_k most relevant 
    database IDs with socres - [(db_id, score)] highest to lowest."""
    qv = embed_query(question)
    scores = []
    for rec in SCHEMAS:
        score = cosine(qv, rec["_norm"])
        scores.append((rec["db_id"], score))
    scores.sort(key=lambda t: t[1], reverse=True)
    return scores[:top_k]

#### Test - Get top databases:

In [111]:
if __name__ == "__main__":
    question = "What are the distinct creation years of the departments managed by a secretary born in state 'Alabama'?"
    
    # Select top 3 databases
    top = select_db(question, top_k=3)
    
    # Print each database with its score and schema
    for db_id, score in top:
        print(f"\nDatabase: '{db_id}'  |  Score: {score:.4f}")
        for rec in SCHEMAS:
            if rec["db_id"] == db_id:
                # Full schema text
                print("\nSchema text:")
                print(rec["text"])
                print("\n")
                break


Database: 'department_management'  |  Score: 0.3950

Schema text:
department(Department_ID, Name, Creation, Ranking, Budget_in_Billions, Num_Employees)
head(head_ID, name, born_state, age)
management(department_ID, head_ID, temporary_acting)



Database: 'election'  |  Score: 0.3278

Schema text:
county(County_Id, County_name, Population, Zip_code)
party(Party_ID, Year, Party, Governor, Lieutenant_Governor, Comptroller, Attorney_General, US_Senate)
election(Election_ID, Counties_Represented, District, Delegate, Party, First_Elected, Committee)



Database: 'scientist_1'  |  Score: 0.2645

Schema text:
Scientists(SSN, Name)
Projects(Code, Name, Hours)
AssignedTo(Scientist, Project)




---

## Testing Database Selector

In [112]:
with open(TEST_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract only db_id and question
questions_db_ids = [{"db_id": item["db_id"], "question": item["question"]} for item in data]

df = pd.DataFrame(questions_db_ids)
df = df[['question', 'db_id']]

# Shuffle the rows
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(df)
# save to json for easier viewing
with open('questions_db_answers.json', 'w', encoding='utf-8') as f:
    json.dump(questions_db_ids, f, ensure_ascii=False, indent=2)


                                                                                                                question  \
0                                                            What are the names of oldest pilots for each type of plane?   
1                   Find the press that published more than two books, and return its name and yearly profit in billion.   
2                           What are the titles and paper ids for papers written in affiliation with Harvard University?   
3                                                                              Which song names have the substring "Is"?   
4                                                            What is id about sailors who do not have boat reservations?   
...                                                                                                                  ...   
2142                                                 Find the manager and sponsor of the team that has the most drivers.   
2143  Fi

In [113]:
if __name__ == "__main__":
    # Test the first 10 questions
    for idx, row in df.head(10).iterrows():
        question = row["question"]
        true_db = row["db_id"]
        
        print(f"\n=== Question {idx+1} ===")
        print(f"Question: {question}")
        print(f"True database: {true_db}\n")
        
        # Select top 3 databases
        top = select_db(question, top_k=3)
        
        # Print each top database with score and schema
        for db_id, score in top:
            print(f"Predicted database: '{db_id}'  |  Score: {score:.4f}")
            for rec in SCHEMAS:
                if rec["db_id"] == db_id:
                    print("\nSchema text:")
                    print(rec["text"])
                    print("\n")
                    break
        
        # Check if true db is in top 3
        top_db_ids = [db for db, _ in top]
        if true_db in top_db_ids:
            print("✅ True database is in top 3 predictions.\n")
        else:
            print("❌ True database is NOT in top 3 predictions.\n")


=== Question 1 ===
Question: What are the names of oldest pilots for each type of plane?
True database: pilot_1

Predicted database: 'aircraft'  |  Score: 0.4141

Schema text:
pilot(Pilot_Id, Name, Age)
aircraft(Aircraft_ID, Aircraft, Description, Max_Gross_Weight, Total_disk_area, Max_disk_Loading)
match(Round, Location, Country, Date, Fastest_Qualifying, Winning_Pilot, Winning_Aircraft)
airport(Airport_ID, Airport_Name, Total_Passengers, %_Change_2007, International_Passengers, Domestic_Passengers, Transit_Passengers, Aircraft_Movements, Freight_Metric_Tonnes)
airport_aircraft(ID, Airport_ID, Aircraft_ID)


Predicted database: 'pilot_record'  |  Score: 0.4110

Schema text:
aircraft(Aircraft_ID, Order_Year, Manufacturer, Model, Fleet_Series, Powertrain, Fuel_Propulsion)
pilot(Pilot_ID, Pilot_name, Rank, Age, Nationality, Position, Join_Year, Team)
pilot_record(Record_ID, Pilot_ID, Aircraft_ID, Date)


Predicted database: 'flight_2'  |  Score: 0.3073

Schema text:
airlines(uid, Airlin

___